# 지점별로 선형모델 생성하기

In [214]:
import numpy as np
from sklearn.model_selection import KFold
import tensorflow as tf
from sklearn.metrics import mean_squared_error
import os
import pandas as pd

In [215]:
tf.__version__

'2.10.1'

## 1-1 입출력 데이터셋 준비

In [216]:
pwd

'd:\\연구\\RnD_Ensemble\\GA'

### 예측 데이터 concat, label 설정

In [217]:
def concat_by_month(directory_path):
    """
    CSV 파일을 읽어와 월별로 데이터를 분류하고 병합
    모든 월의 데이터를 하나의 데이터프레임으로 return, 각 행은 월 정보를 포함

    :param directory_path: CSV 파일이 저장된 디렉토리 경로
    :return: 하나의 데이터프레임으로 병합된 월별 데이터
    """
    # 파일 목록을 읽어온다
    file_list = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

    # 월별 데이터를 저장할 딕셔너리 초기화
    monthly_data = {str(i): [] for i in range(1, 13)}

    # 파일명에서 날짜를 파싱하여 해당 월에 맞는 데이터 프레임 리스트에 추가
    for file_name in file_list:
        date_part = file_name.split('_')[3]  # '20220614' 형식의 문자열 추출
        month = date_part[4:6]  # 월 정보 추출
        
        if month[0] == '0':
            month = month[1]  # '01'과 같은 경우 '1'로 변환
        
        # 파일 읽기
        file_path = os.path.join(directory_path, file_name)
        df = pd.read_csv(file_path)
        df = df[:-1]
        df['Month'] = month  # 월 정보를 데이터프레임에 추가

        # 해당 월에 데이터프레임 추가
        monthly_data[month].append(df)
        
    # 모든 월별 데이터를 하나의 데이터프레임으로 병합
    month_order = ['6', '7', '8', '9', '10', '11', '12', '1', '2', '3', '4', '5']
    all_months_df = pd.concat([pd.concat(monthly_data[month], ignore_index=True) for month in month_order if monthly_data[month]], ignore_index=True)
    
    return all_months_df

In [218]:
month_df = concat_by_month('../../RnD_new/east2')

In [219]:
month_df

,model_fct,model_u,model_v,Month
0,2022-06-01 00:00:00,2.53807,3.43781,6
1,2022-06-01 01:00:00,2.27824,3.29826,6
2,2022-06-01 02:00:00,1.27560,3.40458,6
3,2022-06-01 03:00:00,-0.31523,4.70873,6
4,2022-06-01 04:00:00,-2.59150,8.25900,6
...,...,...,...,...
26131,2023-05-31 19:00:00,0.69256,5.47748,5
26132,2023-05-31 20:00:00,0.19433,6.13060,5
26133,2023-05-31 21:00:00,0.13060,6.30333,5
26134,2023-05-31 22:00:00,0.72848,6.92002,5


### 관측데이터 label 설정

In [220]:
def create_month_column(filename):
    """
    현재 디렉토리에서 지정된 파일을 읽어 'obs_time' 열을 파싱하여 월 정보를 추출하고
    새로운 'Month' 열을 데이터프레임에 추가한 후 반환합니다.

    :param filename: 읽을 파일의 이름
    :return: 'Month' 열이 추가된 데이터프레임
    """
    # 현재 디렉토리 경로 설정
    current_directory = os.getcwd()
    # 파일 경로 완성
    file_path = os.path.join(current_directory, filename)
    
    # 파일이 존재하는지 확인
    if os.path.exists(file_path):
        # 파일 읽기
        df = pd.read_csv(file_path)
        # 'obs_time' 열이 있는지 확인 후 처리
        if 'obs_time' in df.columns:
            # 'obs_time' 열에서 날짜를 파싱하여 월 정보 추출
            df['Month'] = pd.to_datetime(df['obs_time']).dt.month
            return df
        else:
            raise ValueError("The file does not contain an 'obs_time' column.")
    else:
        raise FileNotFoundError("The specified file does not exist in the current directory.")

# 사용 예:
# filename = 'your_data.csv'
# updated_data = create_month_column_from_obs_time(filename)
# print(updated_data.head())  # 'obs_time'과 'Month' 열을 포함하는 데이터프레임 출력


In [221]:
obs_df = create_month_column('../east_obs_22190.csv')

In [222]:
obs_df

,obs_time,obs_u,obs_v,Month
0,2022-06-01 00:00:00,0.328030,0.377355,6
1,2022-06-01 01:00:00,2.139689,2.642296,6
2,2022-06-01 02:00:00,2.074305,2.303749,6
3,2022-06-01 03:00:00,0.136073,2.596437,6
4,2022-06-01 04:00:00,-1.469309,3.827680,6
...,...,...,...,...
8755,2023-05-31 19:00:00,NaN,NaN,5
8756,2023-05-31 20:00:00,NaN,NaN,5
8757,2023-05-31 21:00:00,NaN,NaN,5
8758,2023-05-31 22:00:00,NaN,NaN,5


### 입출력 데이터 프레임 생성

In [223]:
obs_df.columns

Index(['obs_time', 'obs_u', 'obs_v', 'Month'], dtype='object')

In [224]:
month_df.columns

Index(['model_fct', 'model_u', 'model_v', 'Month'], dtype='object')

In [225]:
def create_input_output_data(df_obs, df_pred, start_date, end_date):
    """
    파라미터:
    - df_obs (pd.DataFrame): 관측 데이터를 포함하는 데이터프레임
    - df_pred (pd.DataFrame): 예측 데이터를 포함하는 데이터프레임
    - start_date (str 또는 pd.Timestamp): 데이터 선택을 시작할 날짜 ('YYYY-MM-DD' 형식).
    - end_date (str 또는 pd.Timestamp): 데이터 선택을 종료할 날짜 ('YYYY-MM-DD' 형식).

    반환 값:
    - input_df (pd.DataFrame): 관측값과 예측값이 결합된 (96시간씩) 데이터 프레임
    - output_df (pd.DataFrame): 관측 데이터 (48시간씩) 출력 데이터 프레임
    """
    
    # 시간 열을 datetime 형식으로 변환
    df_obs['obs_time'] = pd.to_datetime(df_obs['obs_time'])
    df_pred['obs_time'] = pd.to_datetime(df_pred['model_fct']) # model_fct를 통일성을 위해 obs_time 이름으로 사용
    
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # input, ouput dataframes
    input_frames = []
    output_frames = []
    
    start_index = 0
    start_index2 = 0
    # 날짜별로 데이터 프레임 구성
    for single_date in pd.date_range(start_date, end_date):
        
        # 관측값은 전일 0h-23h만 선택

        # 시작 인덱스로부터 24개의 행을 슬라이싱
        obs_data_yesterday = df_obs.loc[start_index:start_index + 23, ['obs_time', 'obs_u', 'obs_v']].copy()
        obs_data_yesterday.rename(columns={'obs_u': 'u', 'obs_v': 'v'}, inplace=True)
        
        start_index += 24
        # 'type' 컬럼을 추가하고 'obs'로 설정 (데이터 확인용)
        obs_data_yesterday['type'] = 'obs'
        
        # 예측값은 당일에서 24~71h 후
        # single_date에 해당하는 인덱스를 찾기

        # 시작 인덱스로부터 72개의 행을 슬라이싱
        pred_data_range = df_pred.loc[start_index2:start_index2 + 71, ['obs_time', 'model_u', 'model_v']].copy()
        start_index2 += 72
        
        # 열 이름 변경 및 'type' 컬럼 추가 (데이터 확인용)
        pred_data_range.rename(columns={'model_u': 'u', 'model_v': 'v'}, inplace=True)
        pred_data_range['type'] = 'model'
        if not pred_data_range.empty and len(pred_data_range) >= 25:
            month_value = pred_data_range['obs_time'].iloc[24].month
            pred_data_range['month'] = int(month_value)
            obs_data_yesterday['month'] = int(month_value)
        elif not pred_data_range.empty:
            pred_data_range['month'] = pred_data_range['obs_time'].dt.month.iloc[0]
        else:
            continue  # 데이터가 없으면 이번 루프는 건너뛰기
        
        # input 데이터는 관측값과 예측값을 병합 (관측치 24시간 + 예측치 72시간)
        combined_input = pd.concat([obs_data_yesterday, pred_data_range])

        
        # output 데이터는 다음날 24h부터 내일 모레까지 71h의 관측값
        output_data = df_obs[
            (df_obs['obs_time'] >= single_date + pd.DateOffset(days=1)) & 
            (df_obs['obs_time'] < single_date + pd.DateOffset(days=3))
        ][['obs_time', 'obs_u', 'obs_v']].copy()
        output_data.rename(columns={'obs_u': 'u', 'obs_v': 'v'}, inplace=True)
        output_data['month'] = output_data['obs_time'].iloc[0].month

        
        input_frames.append(combined_input)
        output_frames.append(output_data)

    
    input_df = pd.concat(input_frames, ignore_index=True)
    output_df = pd.concat(output_frames, ignore_index=True)
    
    # 모든 데이터 프레임에서 'obs_time' 열 이름을 'time'으로 변경
    input_df.rename(columns={'obs_time': 'time'}, inplace=True)
    output_df.rename(columns={'obs_time': 'time'}, inplace=True)

    return input_df, output_df


In [226]:
# 입력 및 출력 데이터 생성
input_df, output_df = create_input_output_data(obs_df, month_df, '2022-06-01', '2023-05-31')

In [227]:
input_df.tail(50)

,time,u,v,type,month
34798,2023-05-29 22:00:00,-2.48570,-5.62294,model,5
34799,2023-05-29 23:00:00,-3.16701,-4.96847,model,5
34800,2023-05-30 00:00:00,-3.23537,-3.76208,model,5
34801,2023-05-30 01:00:00,-2.88545,-3.18452,model,5
34802,2023-05-30 02:00:00,-2.11029,-3.34915,model,5
34803,2023-05-30 03:00:00,-1.72445,-4.31804,model,5
34804,2023-05-30 04:00:00,-1.61322,-4.68374,model,5
34805,2023-05-30 05:00:00,-1.59004,-5.50689,model,5
34806,2023-05-30 06:00:00,-2.39533,-5.51229,model,5
34807,2023-05-30 07:00:00,-3.00595,-5.07777,model,5


In [228]:
# 출력 데이터는 관측치 48개씩
output_df[-2000:-1900]

,time,u,v,month
15424,2023-04-19 16:00:00,-4.094381,-0.214577,4
15425,2023-04-19 17:00:00,-4.122834,-0.801398,4
15426,2023-04-19 18:00:00,-4.417322,-0.858640,4
15427,2023-04-19 19:00:00,-3.897624,-0.136108,4
15428,2023-04-19 20:00:00,-3.894655,-0.204110,4
...,...,...,...,...
15519,2023-04-21 15:00:00,-1.147250,4.969287,4
15520,2023-04-21 16:00:00,-2.821369,3.883282,4
15521,2023-04-21 17:00:00,-2.938926,4.045085,4
15522,2023-04-21 18:00:00,-3.523804,2.956823,4


### 96(192), 48(96)개씩 데이터 잘라서 입출력 생성

In [229]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [230]:
# 시간, u,v, month 열만 사용
input = input_df[['time', 'u', 'v', 'month']]
output = output_df[['time', 'u', 'v', 'month']]

In [231]:
input[:26]

,time,u,v,month
0,2022-06-01 00:00:00,0.328030,0.377355,6
1,2022-06-01 01:00:00,2.139689,2.642296,6
2,2022-06-01 02:00:00,2.074305,2.303749,6
3,2022-06-01 03:00:00,0.136073,2.596437,6
4,2022-06-01 04:00:00,-1.469309,3.827680,6
5,2022-06-01 05:00:00,-1.406550,7.976943,6
6,2022-06-01 06:00:00,1.224723,8.714359,6
7,2022-06-01 07:00:00,1.336062,9.506573,6
8,2022-06-01 08:00:00,2.660583,8.702373,6
9,2022-06-01 09:00:00,3.040666,9.945569,6


In [232]:
input.to_csv('input.csv')

In [233]:
output

,time,u,v,month
0,2022-06-02 00:00:00,1.575171,1.062467,6
1,2022-06-02 01:00:00,2.022542,1.469463,6
2,2022-06-02 02:00:00,1.455362,2.737138,6
3,2022-06-02 03:00:00,0.235321,2.689726,6
4,2022-06-02 04:00:00,-0.705993,3.632021,6
...,...,...,...,...
17419,2023-05-31 19:00:00,NaN,NaN,5
17420,2023-05-31 20:00:00,NaN,NaN,5
17421,2023-05-31 21:00:00,NaN,NaN,5
17422,2023-05-31 22:00:00,NaN,NaN,5


In [234]:
# time열을 input, ouput 데이터프레임의 인덱스로 지정
columns = ['u', 'v', 'month']

input_feat = input_df[columns]
input_feat.index = input_df['time']

output_feat = output_df[columns]
output_feat.index = output_df['time']

In [235]:
input_feat[96*3+23:96*3+50]

,u,v,month
time,,,
2022-06-04 23:00:00,-4.77320,3.467933,6
2022-06-04 00:00:00,-2.73018,2.505800,6
2022-06-04 01:00:00,-2.82742,2.753340,6
2022-06-04 02:00:00,-2.80829,3.838190,6
2022-06-04 03:00:00,-2.77163,4.313330,6
2022-06-04 04:00:00,-2.90712,4.316650,6
2022-06-04 05:00:00,-3.37257,4.491090,6
2022-06-04 06:00:00,-3.82049,4.514350,6
2022-06-04 07:00:00,-4.21900,4.446240,6


In [236]:
output_feat

,u,v,month
time,,,
2022-06-02 00:00:00,1.575171,1.062467,6
2022-06-02 01:00:00,2.022542,1.469463,6
2022-06-02 02:00:00,1.455362,2.737138,6
2022-06-02 03:00:00,0.235321,2.689726,6
2022-06-02 04:00:00,-0.705993,3.632021,6
...,...,...,...
2023-05-31 19:00:00,NaN,NaN,5
2023-05-31 20:00:00,NaN,NaN,5
2023-05-31 21:00:00,NaN,NaN,5


In [237]:
# DataFrame을 numpy 배열로 변환
input_array = input_feat.values
output_array = output_feat.values

In [238]:
# 배열별로 결측치 세기
nan_count_in_input = np.sum(np.isnan(input_array))
nan_count_in_output = np.sum(np.isnan(output_array))

In [239]:
nan_count_in_input

162

In [240]:
nan_count_in_output

468

In [241]:
input_array.shape

(34848, 3)

In [242]:
output_array.shape

(17424, 3)

In [243]:
len(input_array)

34848

In [244]:
len(output_array)

17424

In [245]:
input_array[94*3:94*4]

array([[-11.78583   ,  -4.71934   ,   6.        ],
       [-11.13228   ,  -5.37888   ,   6.        ],
       [-10.70349   ,  -6.14476   ,   6.        ],
       [ -8.66154   ,  -7.27779   ,   6.        ],
       [ -8.46229   ,  -7.74794   ,   6.        ],
       [ -8.2997    ,  -7.68315   ,   6.        ],
       [ -2.01309445,   2.98453526,   6.        ],
       [ -2.34861019,   3.4819578 ,   6.        ],
       [ -0.4631035 ,   3.77167538,   6.        ],
       [  0.07853583,   4.49931463,   6.        ],
       [  0.71959854,   4.54336637,   6.        ],
       [ -2.07853411,   5.41476647,   6.        ],
       [ -2.95733868,   5.33518021,   6.        ],
       [ -2.78928053,   5.98163139,   6.        ],
       [ -2.61789856,   6.16738252,   6.        ],
       [ -3.35515742,   4.97422544,   6.        ],
       [ -5.35064274,   4.81774037,   6.        ],
       [ -3.94005377,   3.07830738,   6.        ],
       [ -6.06217783,   3.5       ,   6.        ],
       [ -4.24185147,   3.95559

In [246]:
output_array[:2]

array([[1.57517139, 1.06246652, 6.        ],
       [2.02254249, 1.46946313, 6.        ]])

### 샘플 생성 및 nan 제거

In [247]:
def simple_slice_data(input_data, output_data, input_window, output_window):
    """
    파라미터:
    - input_data (np.array): 모델 입력을 위한 원본 데이터 배열
    - output_data (np.array): 모델 출력(예측 대상)을 위한 원본 데이터 배열
    - input_window (int): 한 번에 입력으로 사용할 데이터 포인트의 수
    - output_window (int): 한 번에 출력으로 사용할 데이터 포인트의 수

    반환 값:
    - X (np.array): 입력 데이터 슬라이스의 배열.
    - y (np.array): 출력 데이터 슬라이스의 배열.
    """
    
    X = []
    y = []

    # 데이터셋의 최대 인덱스를 계산
    max_index = len(output_data) // 48
    print("max_index: {}".format(max_index))
    
    for i in range(max_index):
        
        # 입출력 데이터 슬라이스
        input_slice = input_data[i * input_window: (i + 1) * input_window]
        output_slice = output_data[i * output_window: (i + 1) * output_window]

        X.append(input_slice)
        y.append(output_slice)

    # 배열로 변환
    X = np.array(X)
    y = np.array(y)

    return X, y


In [248]:
# 입력은 96개씩 행이 한 샘플, 출력은 48개씩 행이 한 샘플 (하루치)
input_window = 96
output_window = 48
X, y = simple_slice_data(input_array, output_array, input_window, output_window)

print("X shape:", X.shape)  # (N, 96, 3)
print("y shape:", y.shape)  # (N, 48, 3)

max_index: 363
X shape: (363, 96, 3)
y shape: (363, 48, 3)


In [249]:
# 입출력에 있는 결측치 세기
X_nan = np.sum(np.isnan(X))
y_nan = np.sum(np.isnan(y))

In [250]:
X_nan

162

In [251]:
y_nan

468

In [252]:
len(X)

363

In [253]:
len(y)

363

In [254]:
# 입력 데이터에서 결측치 위치 찾기
nan_indices_input = np.unique(np.where(np.isnan(X))[0])
nan_indices_input

array([ 17,  19,  32,  33,  34,  35,  38,  54,  57,  64,  65,  70,  81,
        83,  86,  98, 155, 176, 197, 202, 218, 362], dtype=int64)

In [255]:
# 출력 데이터에서 결측치 위치 찾기
nan_indices_output = np.unique(np.where(np.isnan(y))[0])
nan_indices_output

array([ 15,  16,  17,  18,  30,  31,  32,  33,  34,  36,  37,  52,  53,
        55,  56,  62,  63,  64,  68,  69,  79,  80,  81,  82,  84,  85,
        96,  97, 153, 154, 174, 175, 195, 196, 200, 201, 216, 217, 360,
       361, 362], dtype=int64)

In [256]:
# 두 배열을 합쳐서 중복된 값 제거
common_nan_indices = np.unique(np.concatenate([nan_indices_input, nan_indices_output]))

In [257]:
nan_indices_input

array([ 17,  19,  32,  33,  34,  35,  38,  54,  57,  64,  65,  70,  81,
        83,  86,  98, 155, 176, 197, 202, 218, 362], dtype=int64)

In [258]:
nan_indices_output

array([ 15,  16,  17,  18,  30,  31,  32,  33,  34,  36,  37,  52,  53,
        55,  56,  62,  63,  64,  68,  69,  79,  80,  81,  82,  84,  85,
        96,  97, 153, 154, 174, 175, 195, 196, 200, 201, 216, 217, 360,
       361, 362], dtype=int64)

In [259]:
# 두 배열을 합쳐서 중복된 값 제거
common_nan_indices = np.unique(np.concatenate([nan_indices_input, nan_indices_output]))

In [260]:
# input, output 샘플에서 통째로 날려야하는 n번째 샘플들
len(common_nan_indices)

56

In [261]:
common_nan_indices

array([ 15,  16,  17,  18,  19,  30,  31,  32,  33,  34,  35,  36,  37,
        38,  52,  53,  54,  55,  56,  57,  62,  63,  64,  65,  68,  69,
        70,  79,  80,  81,  82,  83,  84,  85,  86,  96,  97,  98, 153,
       154, 155, 174, 175, 176, 195, 196, 197, 200, 201, 202, 216, 217,
       218, 360, 361, 362], dtype=int64)

In [262]:
# input보다 큰 샘플 개수 삭제
valid_indices = common_nan_indices[common_nan_indices < len(X)]

In [263]:
len(valid_indices)

56

In [264]:
# input, ouput에서 common_nan_indices에 나열된 인덱스의 샘플들을 삭제
cleaned_X = np.delete(X, valid_indices, axis=0)
cleaned_y = np.delete(y, valid_indices, axis=0)

# 제거되고 남은 샘플 개수 확인
print(cleaned_X.shape)
print(cleaned_y.shape)

(307, 96, 3)
(307, 48, 3)


In [265]:
# 배열 전체에서 NaN 값 확인
nan_exists = np.isnan(cleaned_y).any()

In [266]:
nan_exists

False

In [267]:
print(cleaned_X.shape)

(307, 96, 3)


In [268]:
cleaned_X[270]

array([[ -3.8904998 ,   0.27205025,   4.        ],
       [ -4.34666622,  -1.1646857 ,   4.        ],
       [ -3.38011819,  -1.50492558,   4.        ],
       [ -3.26690609,  -1.73704478,   4.        ],
       [ -3.07426458,  -2.23358396,   4.        ],
       [ -5.03324666,  -2.45487842,   4.        ],
       [ -4.90243465,  -1.40575051,   4.        ],
       [ -4.89925371,  -0.08551679,   4.        ],
       [ -5.39259949,   0.28261416,   4.        ],
       [ -6.53657953,   1.87433402,   4.        ],
       [ -5.23081084,   1.69959347,   4.        ],
       [ -4.67653718,   2.7       ,   4.        ],
       [ -4.04457321,   1.97267016,   4.        ],
       [ -4.041159  ,   3.27246603,   4.        ],
       [ -3.21468924,   3.69807694,   4.        ],
       [ -2.83194176,   3.49715683,   4.        ],
       [ -2.02856766,   4.35027738,   4.        ],
       [ -1.06034962,   4.98855276,   4.        ],
       [ -1.32877308,   4.08954302,   4.        ],
       [ -1.14336288,   3.51890

In [269]:
cleaned_y.shape

(307, 48, 3)

In [270]:
import numpy as np
import pandas as pd

# cleaned_X_reshaped와 cleaned_y_reshaped 배열이 주어졌다고 가정
# 예시 데이터 생성을 위한 코드는 생략합니다. 실제 데이터 사용을 가정

# 각 시간 단계를 별도의 행으로 변환하고, 열은 u, v, month로 설정
# cleaned_X_reshaped.reshape(-1, 3)는 모든 시간 단계를 행으로, 특성을 열로 펼칩니다.
df_X = pd.DataFrame(cleaned_X.reshape(-1, 3), columns=['u', 'v', 'month'])
df_y = pd.DataFrame(cleaned_y.reshape(-1, 3), columns=['u', 'v', 'month'])

# 'month' 열을 정수형으로 변환
df_X['month'] = df_X['month'].astype(int)
df_y['month'] = df_y['month'].astype(int)

# DataFrame을 CSV 파일로 저장
df_X.to_csv("X.csv", index=False)
df_y.to_csv("y.csv", index=False)

# 저장된 파일 확인 메시지
print("Data has been saved to cleaned_X_reshaped.csv and cleaned_y_reshaped.csv.")


Data has been saved to cleaned_X_reshaped.csv and cleaned_y_reshaped.csv.


## 월별인덱스 확인 및 정규화

In [271]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input, Reshape

In [272]:
# 월별로 인덱스를 분할할 리스트 초기화
monthly_indices = [[] for _ in range(12)]

total = 0
# 월별로 인덱스를 분할
for month in range(1, 13):
    indices = [i for i in range(cleaned_y.shape[0]) if int(cleaned_y[i, 0, 2]) == month]
    
    # 중복 제거 및 리스트로 변환
    total += len(indices)
    print("{0}월의 인덱스: {1}".format(month, indices))
    print("{0}월의 인덱스 개수: {1}".format(month,len(indices)))
    print('================================')
    monthly_indices[month-1].append(indices)

print("전체 index 개수: {}개".format(total))

# K-폴드 교차 검증 설정
kf = KFold(n_splits=12)

1월의 인덱스: [163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190]
1월의 인덱스 개수: 28
2월의 인덱스: [191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218]
2월의 인덱스 개수: 28
3월의 인덱스: [219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249]
3월의 인덱스 개수: 31
4월의 인덱스: [250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279]
4월의 인덱스 개수: 30
5월의 인덱스: [280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306]
5월의 인덱스 개수: 27
6월의 인덱스: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
6월의 인덱스 개수: 24
7월의 인덱스: [24, 25, 26, 27, 28, 29, 30, 31, 32

In [273]:
cleaned_X.shape

(307, 96, 3)

In [274]:
cleaned_y.shape

(307, 48, 3)

## feature 4로 X 분리

In [275]:
# 각 24시간 구간을 4개의 feature로 나누기 위해 reshape
# (batch_size, total_time_steps, features) -> (batch_size, 4, time_steps_per_feature, original_features)
reshaped_X = cleaned_X.reshape((cleaned_X.shape[0], 4, 24, cleaned_X.shape[2]))

# 각 시간 구간별로 feature명을 지정하고 데이터프레임으로 변환 후 numpy 배열로 변환
numpy_arrays = []
for i in range(reshaped_X.shape[1]):  # 4개의 24시간 구간
    data = reshaped_X[:, i, :, :].reshape(-1, reshaped_X.shape[3])
    df = pd.DataFrame(data, columns=[f"feature_{j+1}" for j in range(reshaped_X.shape[3])])
    numpy_array = df.to_numpy()  # 데이터프레임을 numpy 배열로 변환
    numpy_arrays.append(numpy_array)
    print(f"Numpy array for time segment {i+1}:")
    print(numpy_array)
    print("\n")

# 각 시간 구간에 대한 numpy 배열
feature1, feature2, feature3, feature4 = numpy_arrays

# feature들을 24시간씩 잘라서 (sample_size, 24, 3) 형태로 변환
def reshape_features(feature):
    num_samples = feature.shape[0] // 24
    reshaped_feature = feature[:num_samples * 24].reshape(num_samples, 24, feature.shape[1])
    return reshaped_feature

feature1 = reshape_features(feature1)
feature2 = reshape_features(feature2)
feature3 = reshape_features(feature3)
feature4 = reshape_features(feature4)

Numpy array for time segment 1:
[[ 0.32802951  0.37735479  6.        ]
 [ 2.13968933  2.64229627  6.        ]
 [ 2.07430488  2.30374896  6.        ]
 ...
 [-2.89360893 -3.70365054  5.        ]
 [-3.34415171 -3.01108773  5.        ]
 [-3.30896308 -3.1954285   5.        ]]


Numpy array for time segment 2:
[[2.53807 3.43781 6.     ]
 [2.27824 3.29826 6.     ]
 [1.2756  3.40458 6.     ]
 ...
 [1.6786  6.024   5.     ]
 [1.36807 6.95106 5.     ]
 [1.85124 6.70546 5.     ]]


Numpy array for time segment 3:
[[2.2352  4.22362 6.     ]
 [2.32447 4.15052 6.     ]
 [1.81598 3.44279 6.     ]
 ...
 [0.75976 6.82421 5.     ]
 [0.68445 6.40589 5.     ]
 [0.28123 6.26824 5.     ]]


Numpy array for time segment 4:
[[ 1.44297 -6.48367  6.     ]
 [ 1.81119 -7.09836  6.     ]
 [ 1.76497 -7.08175  6.     ]
 ...
 [ 3.0134   5.69878  5.     ]
 [ 3.90558  3.02151  5.     ]
 [ 3.83722 -2.08608  5.     ]]




In [276]:
# 마지막 차원의 크기를 2로 변경 (예시로 첫 번째와 두 번째 feature만 사용)
reshaped_X = reshaped_X[:, :, :, :2]
reshaped_X.shape

(307, 4, 24, 2)

In [277]:
reshaped_X

array([[[[ 0.32802951,  0.37735479],
         [ 2.13968933,  2.64229627],
         [ 2.07430488,  2.30374896],
         ...,
         [ 4.69928417,  0.08202631],
         [ 4.4432588 ,  1.19056761],
         [ 2.15710571,  1.05209075]],

        [[ 2.53807   ,  3.43781   ],
         [ 2.27824   ,  3.29826   ],
         [ 1.2756    ,  3.40458   ],
         ...,
         [ 3.7033    ,  4.31001   ],
         [ 3.61722   ,  3.8033    ],
         [ 2.9031    ,  3.36969   ]],

        [[ 2.2352    ,  4.22362   ],
         [ 2.32447   ,  4.15052   ],
         [ 1.81598   ,  3.44279   ],
         ...,
         [ 1.98494   , -6.78437   ],
         [ 1.7841    , -6.36904   ],
         [ 1.71874   , -6.34578   ]],

        [[ 1.44297   , -6.48367   ],
         [ 1.81119   , -7.09836   ],
         [ 1.76497   , -7.08175   ],
         ...,
         [-4.26363   ,  0.22479   ],
         [-4.01496   , -0.41648   ],
         [-4.31783   , -0.04434   ]]],


       [[[ 1.57517139,  1.06246652],
         

In [278]:
X = np.concatenate((feature1, feature2, feature3, feature4), axis=2)

In [279]:
X.shape

(307, 24, 12)

In [280]:
X = np.delete(X, [2,5,8,11], axis=2)

In [281]:
X.shape

(307, 24, 8)

In [282]:
X

array([[[ 0.32802951,  0.37735479,  2.53807   , ...,  4.22362   ,
          1.44297   , -6.48367   ],
        [ 2.13968933,  2.64229627,  2.27824   , ...,  4.15052   ,
          1.81119   , -7.09836   ],
        [ 2.07430488,  2.30374896,  1.2756    , ...,  3.44279   ,
          1.76497   , -7.08175   ],
        ...,
        [ 4.69928417,  0.08202631,  3.7033    , ..., -6.78437   ,
         -4.26363   ,  0.22479   ],
        [ 4.4432588 ,  1.19056761,  3.61722   , ..., -6.36904   ,
         -4.01496   , -0.41648   ],
        [ 2.15710571,  1.05209075,  2.9031    , ..., -6.34578   ,
         -4.31783   , -0.04434   ]],

       [[ 1.57517139,  1.06246652,  2.81065   , ..., -5.97862   ,
         -3.88266   ,  1.95756   ],
        [ 2.02254249,  1.46946313,  1.36965   , ..., -6.33415   ,
         -4.16002   ,  2.08881   ],
        [ 1.45536184,  2.73713754, -0.43797   , ..., -6.47038   ,
         -4.75778   ,  2.35296   ],
        ...,
        [ 1.62187646, -8.34383106,  1.08432   , ...,  

In [283]:
cleaned_y.shape

(307, 48, 3)

In [284]:
# 3번째 열을 제거
y_processed = cleaned_y[:, :, :2]

In [285]:
y_processed.shape

(307, 48, 2)

In [286]:
X.shape

(307, 24, 8)

# 2. 모델 성능 측정

## 시간대별 (24~71h) 데이터 정리 및 저장

In [287]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, SimpleRNN, LSTM, GRU, PReLU, Dense, Dropout, BatchNormalization, Conv1D, MaxPooling1D, UpSampling1D, Flatten, Dense, Reshape
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad
import itertools
from tensorflow.keras.initializers import glorot_normal
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, StandardScaler
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import LeakyReLU ,Lambda, Activation, Bidirectional
from keras.optimizers import Adam
from keras.utils import plot_model
from tensorflow.keras.models import Model


In [288]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import LinearSVR
import warnings

In [289]:
from sklearn.svm import SVR
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dropout, Dense, Reshape

# 첫 번째 기본 모델
def build_regression_model(input_steps, features, output_steps, output_features):
    model = Sequential([
        Conv1D(filters=512, kernel_size=2, activation='tanh', padding='same', input_shape=(input_steps, features)),
        Dropout(0.4),
        Conv1D(filters=512, kernel_size=2, activation='tanh', padding='same'),
        Bidirectional(GRU(units=512, return_sequences=True, activation='tanh')),
        Dropout(0.4),
        Flatten(),
        Dense(units=1024, activation='selu'),
        Dropout(0.4),
        Dense(units=output_steps * output_features, activation='linear'),
        Reshape((output_steps, output_features))
    ])
    
    optimizer = SGD(learning_rate=0.01, momentum=0.8, nesterov=True)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
    
    return model

# 두 번째 기본 모델
def build_regression_model2(input_steps, features, output_steps, output_features, lstm_units=512):
    # 입력 레이어
    inputs = Input(shape=(input_steps, features))
    
    # LSTM 레이어로 변경
    x = LSTM(units=lstm_units, activation='tanh', return_sequences=False)(inputs)  # Conv1D 대체
    
    # 드롭아웃 적용 (과적합 방지)
    x = Dropout(0.4)(x)
    
    # 완전 연결(Dense) 레이어
    x = Dense(units=output_steps * output_features, activation='linear')(x)
    
    # 출력 형태 변환
    outputs = Reshape((output_steps, output_features))(x)
    
    # 모델 생성
    model = Model(inputs=inputs, outputs=outputs)
    
    # 옵티마이저 설정 (SGD)
    optimizer = SGD(learning_rate=0.01, momentum=0.8, nesterov=True)
    
    # 모델 컴파일
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
    
    return model


def build_regression_model3(input_steps, features, output_steps, output_features):
    model = Sequential([
                Input(shape=(input_steps, features)),
                Bidirectional(GRU(units=1024, return_sequences=False)),  # 양방향 LSTM 레이어
                Dropout(0.6),  # 드롭아웃 레이어
                Dense(units=1024, activation='elu'),  # 완전 연결 레이어 추가
                Dropout(0.4), 
                Dense(units=1024, activation='elu'),
                Dropout(0.4),
                Dense(units=1024, activation='elu'),
                Dropout(0.4),
                Dense(units=output_steps * output_features, activation='linear'),  # 출력 레이어
                Reshape((output_steps, output_features))  # 출력 형상 조정
            ])
    
    optimizer = SGD(learning_rate=0.01, momentum=0.8, nesterov=True)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])

    return model

def create_and_compile_model(input_shape=(192,), learning_rate=0.01, momentum=0.8, dropout_rate=0.6):
    """
    선형 모델을 생성하고 컴파일하는 함수

    Parameters:
    - input_shape: 입력 데이터의 형태 (기본값은 (192,))
    - learning_rate: SGD 옵티마이저의 학습률 (기본값은 0.01)
    - momentum: SGD 옵티마이저의 모멘텀 값 (기본값은 0.8)
    - dropout_rate: Dropout 레이어의 비율 (기본값은 0.6)

    Returns:
    - model: 컴파일된 Keras 모델
    """
    # 모델 생성
    model = Sequential([
        Input(shape=input_shape),
        Dense(1024, activation='selu'),
        Dropout(dropout_rate),
        Dense(96, activation='linear')
    ])

    # 옵티마이저 설정
    optimizer = SGD(learning_rate=learning_rate, momentum=momentum, nesterov=True)

    # 모델 컴파일
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
    
    return model



In [290]:
import numpy as np
import pandas as pd

def load_scaled_data_from_csv(file_path, input_steps, features):
    """
    CSV에서 2D 데이터를 로드한 뒤 3D로 변환합니다.
    """
    data_2d = pd.read_csv(file_path, header=None).values  # CSV 읽기
    data_3d = data_2d.reshape(-1, input_steps, features)  # 2D -> 3D 변환
    print(f"Data loaded from {file_path} and reshaped to {data_3d.shape}")
    return data_3d

In [291]:
def train_and_predict_svr_3d(X_train, y_train, X_val, X_test):
    """
    3차원 데이터를 2차원으로 변환하여 SVR 모델 학습 및 예측을 수행하고,
    다시 3차원 형태로 복원하는 함수
    """
    # 데이터 3차원 -> 2차원 변환
    n_samples, n_time, n_features = X_train.shape
    
    # 올바르게 2D 변환
    X_train_2d = X_train.reshape(n_samples,192)
    y_train_2d = y_train.reshape(n_samples,96)
    X_val_2d = X_val.reshape(X_val.shape[0],X_val.shape[1]*X_val.shape[2])
    X_test_2d = X_test.reshape(X_test.shape[0],X_test.shape[1]*X_test.shape[2])

    # MultiOutputRegressor를 사용하여 다중 출력 SVR 모델 생성
    svr_model = MultiOutputRegressor(SVR(kernel='rbf', C=1.0, epsilon=0.1))
    print(X_train_2d.shape)
    # 모델 학습
    svr_model.fit(X_train_2d, y_train_2d)

    # 예측 수행
    y_val_pred_2d = svr_model.predict(X_val_2d)
    y_test_pred_2d = svr_model.predict(X_test_2d)

    # 예측 결과를 3차원으로 복원
    y_val_pred_3d = y_val_pred_2d.reshape(-1,48, 2)
    y_test_pred_3d = y_test_pred_2d.reshape(-1,48, 2)

    return svr_model, y_test_pred_3d, y_val_pred_3d


In [292]:
def train_and_predict_nn_3d(X_train, y_train, X_val, X_test, input_shape, learning_rate=0.01, momentum=0.8, dropout_rate=0.6):
    """
    3차원 데이터를 입력받아 NN 모델을 학습하고 예측을 수행한 후,
    3차원 형태로 반환하는 함수

    Parameters:
    - X_train: 훈련 데이터 (3차원: 샘플수, 시간, 특성수)
    - y_train: 훈련 라벨 (3차원: 샘플수, 시간, 특성수)
    - X_val: 검증 데이터 (3차원: 샘플수, 시간, 특성수)
    - X_test: 테스트 데이터 (3차원: 샘플수, 시간, 특성수)
    - input_shape: 입력 데이터의 형태 (3차원: 샘플수, 시간, 특성수)
    - learning_rate: SGD 옵티마이저의 학습률 (기본값은 0.01)
    - momentum: SGD 옵티마이저의 모멘텀 값 (기본값은 0.8)
    - dropout_rate: Dropout 레이어의 비율 (기본값은 0.6)

    Returns:
    - results: NN 모델의 예측 결과
    - trained_nn_model: 학습된 NN 모델
    """
    # 데이터 3차원 -> 2차원 변환
    n_samples, n_time, n_features = X_train.shape

    X_train_2d = X_train.reshape(n_samples, -1)
    y_train_2d = y_train.reshape(n_samples, -1)
    X_val_2d = X_val.reshape(X_val.shape[0], -1)
    X_test_2d = X_test.reshape(X_test.shape[0], -1)

    nn_model = Sequential([
        Input(shape=(192,)),
        Dense(1024, activation='elu'),
        Dropout(dropout_rate),
        Dense(96, activation='linear')
    ])

    optimizer = SGD(learning_rate=learning_rate, momentum=momentum, nesterov=True)
    nn_model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])

    # Early Stopping 설정
    early_stopping = EarlyStopping(
        monitor='loss', 
        patience=30, 
        verbose=1, 
        restore_best_weights=True
    )

    # NN 모델 학습
    nn_model.fit(
        X_train_2d, y_train_2d,
        epochs=300, 
        batch_size=128, 
        verbose=1,
        callbacks=[early_stopping]
    )

    # NN 예측 수행
    y_val_pred_2d = nn_model.predict(X_val_2d)
    y_test_pred_2d = nn_model.predict(X_test_2d)

    # NN 예측 결과를 3차원으로 복원
    y_val_pred_3d = y_val_pred_2d.reshape(-1, 48, 2)
    y_test_pred_3d = y_test_pred_2d.reshape(-1, 48, 2)

    print(y_val_pred_3d.shape)
    print(y_test_pred_3d.shape)
    return nn_model, y_val_pred_3d, y_test_pred_3d

In [293]:
monthly_indices

[[[163,
   164,
   165,
   166,
   167,
   168,
   169,
   170,
   171,
   172,
   173,
   174,
   175,
   176,
   177,
   178,
   179,
   180,
   181,
   182,
   183,
   184,
   185,
   186,
   187,
   188,
   189,
   190]],
 [[191,
   192,
   193,
   194,
   195,
   196,
   197,
   198,
   199,
   200,
   201,
   202,
   203,
   204,
   205,
   206,
   207,
   208,
   209,
   210,
   211,
   212,
   213,
   214,
   215,
   216,
   217,
   218]],
 [[219,
   220,
   221,
   222,
   223,
   224,
   225,
   226,
   227,
   228,
   229,
   230,
   231,
   232,
   233,
   234,
   235,
   236,
   237,
   238,
   239,
   240,
   241,
   242,
   243,
   244,
   245,
   246,
   247,
   248,
   249]],
 [[250,
   251,
   252,
   253,
   254,
   255,
   256,
   257,
   258,
   259,
   260,
   261,
   262,
   263,
   264,
   265,
   266,
   267,
   268,
   269,
   270,
   271,
   272,
   273,
   274,
   275,
   276,
   277,
   278,
   279]],
 [[280,
   281,
   282,
   283,
   284,
   285,
   286,


In [294]:
print(f"4월 데이터 크기 확인: {monthly_indices[3]}")


4월 데이터 크기 확인: [[250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279]]


In [295]:
# 빈 리스트 제거
for i in range(len(monthly_indices) - 1, -1, -1):  # 역순으로 순회
    if isinstance(monthly_indices[i], list) and all(len(inner) == 0 for inner in monthly_indices[i]):
        del monthly_indices[i]


In [296]:
len(monthly_indices)

12

In [297]:
from sklearn.preprocessing import MaxAbsScaler

from sklearn.preprocessing import MaxAbsScaler
import numpy as np
import itertools
import csv
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping
import os

def ensemble_models(
    cleaned_X, cleaned_y, unique_monthly_indices,
    input_steps, features, output_steps, output_features,
    CONV_WIDTH, STRIDE, output_dir
):
    results_dict = {i: {'predictions': [], 'true_values': []} for i in range(output_steps)}
    fold_rmse_values = []

    early_stopping = EarlyStopping(
        monitor='loss',
        patience=30,
        verbose=1,
        mode='min',
        restore_best_weights=True
    )

    os.makedirs(output_dir, exist_ok=True)

    for month in range(12):
        print(f"Processing Month {month + 1}")

        # 데이터 분할
        try:
            test_indices = unique_monthly_indices[month]
            validation_indices = unique_monthly_indices[(month + 1) % 12]
            train_indices = []

            for month_idx in range(12):
                if month_idx != month and month_idx != (month + 1) % 12:
                    train_indices.extend(unique_monthly_indices[month_idx])

            train_flatten_indices = list(itertools.chain(*train_indices))
            validation_flatten_indices = list(itertools.chain(*validation_indices))
            test_flatten_indices = list(itertools.chain(*test_indices))

            # 빈 데이터 확인
            if not train_flatten_indices or not validation_flatten_indices or not test_flatten_indices:
                print(f"{month + 1}월 데이터 확인: train={len(train_flatten_indices)}, val={len(validation_flatten_indices)}, test={len(test_flatten_indices)}")
                continue

            X_train = cleaned_X[train_flatten_indices, :, :]
            y_train = cleaned_y[train_flatten_indices, :, :]
            X_val = cleaned_X[validation_flatten_indices, :, :]
            y_val = cleaned_y[validation_flatten_indices, :, :]
            X_test = cleaned_X[test_flatten_indices, :, :]
            y_test = cleaned_y[test_flatten_indices, :, :]
        except IndexError:
            print(f"{month + 1}월: 데이터가 부족하여 건너뜁니다.")
            continue


        scaler = MaxAbsScaler()
        try:
            X_train_scaled_3d = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1)).reshape(-1, input_steps, features)
            X_val_scaled_3d = scaler.transform(X_val.reshape(X_val.shape[0], -1)).reshape(-1, input_steps, features)
            X_test_scaled_3d = scaler.transform(X_test.reshape(X_test.shape[0], -1)).reshape(-1, input_steps, features)
        except ValueError as e:
            print(f"스케일링 중 오류 발생: {e}")
            continue

        models = [
            build_regression_model2(input_steps, features, output_steps, output_features, 512),
        ]


        # 첫 번째 학습: 검증 데이터에 대한 예측 수행
        for i, model in enumerate(models):
            print(f"Training Model {i + 1} (Validation Phase)")
            model.fit(
                X_train_scaled_3d, y_train,
                epochs=300,
                batch_size=128,
                callbacks=[early_stopping],
                verbose=1
            )

        model_predictions_val = [model.predict(X_val_scaled_3d) for model in models]


        output_csv_path_val = f"{output_dir}/val_month_{month + 1}_model_2_results.csv"
        with open(output_csv_path_val, "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([
                "Sample Index", "Time Step",
                "Model 2 Val Pred U", "Model 2 Val Pred V",
                "True U", "True V"
            ])

            for j in range(X_val_scaled_3d.shape[0]):
                y_val_sample = y_val[j, :, :]
                val_pred_samples = [pred[j, :, :] for pred in model_predictions_val]

                for t in range(output_steps):
                    writer.writerow([
                        j, t,
                        val_pred_samples[0][t, 0], val_pred_samples[0][t, 1],
                        y_val_sample[t, 0], y_val_sample[t, 1]
                    ])

        # 두 번째 학습: 테스트 데이터에 대한 예측 수행
        second_train_indices = []
        for month_idx in range(12):
            if month_idx != month:
                second_train_indices.extend(unique_monthly_indices[month_idx])

        second_train_flatten_indices = list(itertools.chain(*second_train_indices))

        X_train_second = cleaned_X[second_train_flatten_indices, :, :]
        y_train_second = cleaned_y[second_train_flatten_indices, :, :]

        X_train_scaled_3d_second = scaler.fit_transform(X_train_second.reshape(X_train_second.shape[0], -1)).reshape(-1, input_steps, features)

        for i, model in enumerate(models):
            print(f"Training Model {i + 1} (Test Phase)")
            model.fit(
                X_train_scaled_3d_second, y_train_second,
                epochs=300,
                batch_size=128,
                callbacks=[early_stopping],
                verbose=1
            )

        model_predictions_test_second = [model.predict(X_test_scaled_3d) for model in models]

        output_csv_path_test = f"{output_dir}/test_month_{month + 1}_model_2_results.csv"
        with open(output_csv_path_test, "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([
                "Sample Index", "Time Step",
                "Model 2 Test Pred U", "Model 2 Test Pred V",
                "True U", "True V"
            ])

            for j in range(X_test_scaled_3d.shape[0]):
                y_test_sample = y_test[j, :, :]
                test_pred_samples = [pred[j, :, :] for pred in model_predictions_test_second]

                for t in range(output_steps):
                    writer.writerow([
                        j, t,
                        test_pred_samples[0][t, 0], test_pred_samples[0][t, 1],
                        y_test_sample[t, 0], y_test_sample[t, 1]
                    ])

        print(f"{month + 1}월 결과 완료")


ensemble_models(X, y_processed, monthly_indices,24,8,48,2,2,1, './ga_e2')

Processing Month 1
Training Model 1 (Validation Phase)
Epoch 1/300
2/2 [==============================] - 1s 25ms/step - loss: 22.6539 - mse: 22.6539
Epoch 2/300
2/2 [==============================] - 0s 23ms/step - loss: 22.6422 - mse: 22.6422
Epoch 3/300
2/2 [==============================] - 0s 22ms/step - loss: 22.6132 - mse: 22.6132
Epoch 4/300
2/2 [==============================] - 0s 25ms/step - loss: 22.5908 - mse: 22.5908
Epoch 5/300
2/2 [==============================] - 0s 21ms/step - loss: 22.5581 - mse: 22.5581
Epoch 6/300
2/2 [==============================] - 0s 20ms/step - loss: 22.5251 - mse: 22.5251
Epoch 7/300
2/2 [==============================] - 0s 20ms/step - loss: 22.4970 - mse: 22.4970
Epoch 8/300
2/2 [==============================] - 0s 20ms/step - loss: 22.4565 - mse: 22.4565
Epoch 9/300
2/2 [==============================] - 0s 21ms/step - loss: 22.4148 - mse: 22.4148
Epoch 10/300
2/2 [==============================] - 0s 9ms/step - loss: 22.3784 - mse: 22.

### RMSE 계산

In [298]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

def calculate_model_avg_rmse(base_dir, months, models):
    # 모델별 월별 RMSE 저장
    model_avg_rmse = {model: [] for model in models}

    # 월별 처리
    for month in months:
        true_values = None

        for model in models:
            # 파일 처리
            file_name = f"test_month_{month}_model_{model}_results.csv"
            model_path = os.path.join(base_dir, f"model{model}", file_name)
            if not os.path.exists(model_path):
                print(f"{model_path} 파일이 존재하지 않습니다.")
                continue

            # CSV 파일 읽기
            data = pd.read_csv(model_path)

            # 모델 예측값(U, V) 추가
            u_col = f"Model {model} Test Pred U"
            v_col = f"Model {model} Test Pred V"
            model_predictions = data[[u_col, v_col]].values.reshape(-1, 48, 2)  # (samples, 48, 2)

            # 실제값 저장
            if true_values is None:
                true_u = data["True U"].values.reshape(-1, 48, 1)  # (samples, 48, 1)
                true_v = data["True V"].values.reshape(-1, 48, 1)  # (samples, 48, 1)
                true_values = np.concatenate([true_u, true_v], axis=2)  # (samples, 48, 2)

            # 월별 RMSE 계산
            mse = mean_squared_error(
                true_values.reshape(-1, 2),  # 전체 샘플을 2차원으로 펼침
                model_predictions.reshape(-1, 2),  # 전체 샘플을 2차원으로 펼침
                multioutput='uniform_average'
            )
            rmse = np.sqrt(mse)
            model_avg_rmse[model].append(rmse)

    # 모델별 평균 RMSE 출력
    print("\n--- 모델별 월별 평균 RMSE ---")
    for model, monthly_rmses in model_avg_rmse.items():
        model_monthly_mean_rmse = np.mean(monthly_rmses) if monthly_rmses else float('nan')
        print(f"Model {model} 월별 평균 RMSE: {model_monthly_mean_rmse:.4f}")

# 베이스 디렉토리와 월, 모델 설정
base_dir = "./ga_e1"
months = range(1, 13)  # 1월부터 12월까지
models = [2]

# 모델별 평균 RMSE 계산
calculate_model_avg_rmse(base_dir, months, models)



--- 모델별 월별 평균 RMSE ---
Model 2 월별 평균 RMSE: 3.7559
